<a href="https://colab.research.google.com/github/ChriRRojas/Biomedica-/blob/main/Chambitas_Moi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

># **Import Packages**   🧮

In [1]:
!pip install cimcb


In [2]:
!pip uninstall -y scipy


Found existing installation: scipy 1.9.3
Uninstalling scipy-1.9.3:
  Successfully uninstalled scipy-1.9.3


In [4]:
!pip install scipy==1.9.3


In [6]:
!pip install cimcb



In [7]:
import numpy as np
import pandas as pd
import cimcb as cb
from sklearn.model_selection import train_test_split

print('All packages successfully loaded')

All packages successfully loaded


># **Load Data & Peak Sheet**   ❗

> **The data used was MTBLS90**


In [8]:
# prompt: home = 'data/'
# file = 'MTBLS90.xlsx'
# DataTable,PeakTable = cb.utils.load_dataXL(home + file, DataSheet='Data', PeakSheet='Peak')     tengo este codigo pero en ves de llamar mi data en local lo quiero hacer en drive, me ayudas?

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [16]:
home = '/content/drive/MyDrive/Chambitas Moi'
file = 'ComparisonBinaryML/MTBLS90.xlsx' # Cambio: Se eliminó la ruta redundante de 'file'
DataTable, PeakTable = cb.utils.load_dataXL(home + '/' + file, DataSheet='Data', PeakSheet='Peak') # Cambio: Se cambió '+' a '/' para una concatenación de ruta adecuada

Loadings PeakFile: Peak


/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


Loadings DataFile: Data
Data Table & Peak Table is suitable.
TOTAL SAMPLES: 968 TOTAL PEAKS: 189
Done!


/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


># **"The damn *ANN LIN SING model*, first practice." 😅**

># **Extract X & Y** ⏬

In [ ]:
# Extract PeakList
PeakList = PeakTable['Name']

# Select Subset of Data
DataTable2 = DataTable[(DataTable.Class == 1) | (DataTable.Class == 0)]

# Create a Binary Y Vector
Outcomes = DataTable2['Class']
Y = Outcomes.values

# Split Data into Train (2/3) and Test (1/3)
DataTrain, DataTest, YTrain, YTest = train_test_split(DataTable2, Y, test_size=1/3, stratify=Y, random_state=8)

# Extract Train Data
XTrain = DataTrain[PeakList]
XTrainLog = np.log(XTrain)
XTrainScale, mu, sigma = cb.utils.scale(XTrainLog, method='auto', return_mu_sigma=True)
XTrainKnn = cb.utils.knnimpute(XTrainScale, k=3)

# Extract Test Data
XTest = DataTest[PeakList]
XTestLog = np.log(XTest)
XTestScale = cb.utils.scale(XTestLog, method='auto', mu=mu, sigma=sigma)
XTestKnn = cb.utils.knnimpute(XTestScale, k=3)

># **Hyperparameters Optimisation** 🙏

In [ ]:
# Parameter Dictionary
lr = [0.0001,0.0005,0.001,0.005,0.01,0.05,0.1,1]
neurons = [2, 3, 4, 5, 6]

param_dict = dict(learning_rate=lr,
                  n_neurons=neurons,
                  epochs=1000,
                  momentum=0.5,
                  decay=0,
                  loss='binary_crossentropy')


# Initialise
cv = cb.cross_val.KFold(model=cb.model.NN_LinearSigmoid,
                        X=XTrainKnn,
                        Y=YTrain,
                        param_dict=param_dict,
                        folds=5,
                        n_mc=10)


# Run and Plot
cv.run()
cv.plot(metric='auc', color_beta=[5,5,5])
cv.plot(metric='r2q2', color_beta=[5,5,5])

In [ ]:
# Parameter Dictionary
lr = [0.001,0.002,0.003,0.004,0.005,0.006,0.007,0.008,0.009,0.01]

param_dict = dict(learning_rate=lr,
                  n_neurons=3,
                  epochs=1000,
                  momentum=0.5,
                  decay=0,
                  loss='binary_crossentropy')

# Initialise
cv = cb.cross_val.KFold(model=cb.model.NN_LinearSigmoid,
                        X=XTrainKnn,
                        Y=YTrain,
                        param_dict=param_dict,
                        folds=5,
                        n_mc=10)

# Run and Plot
cv.run()
cv.plot(metric='auc')
cv.plot(metric='r2q2')

># **Build Model & Evaluate** 😋

In [ ]:
# Build Model
model = cb.model.NN_LinearSigmoid(learning_rate=0.003,
                                  n_neurons=3,
                                  epochs=1000,
                                  momentum=0.5,
                                  decay=0,
                                  loss='binary_crossentropy')
YPredTrain = model.train(XTrainKnn, YTrain)
YPredTest = model.test(XTestKnn)

# Put YTrain and YPredTrain in a List
EvalTrain = [YTrain, YPredTrain]

# Put YTest and YPrestTest in a List
EvalTest = [YTest, YPredTest]

# Evaluate Model (include Test Dataset)
model.evaluate(testset=EvalTest)

># **Bootstrap Evaluation**  ✅

In [ ]:
# Extract X Data
XBoot = DataTable2[PeakList]
XBootLog = np.log(XBoot)
XBootScale = cb.utils.scale(XBootLog, method='auto')
XBootKnn = cb.utils.knnimpute(XBootScale, k=3)
YPredBoot = model.train(XBootKnn, Y)

# Build Boostrap Models
bootmodel = cb.bootstrap.Per(model, bootnum=100)
bootmodel.run()

# Boostrap Evaluate Model (include Test Dataset)
bootmodel.evaluate(trainset=EvalTrain, testset=EvalTest)

># **Eh, this part doesn’t even matter.** 😏

In [ ]:
home = 'results/'
file = 'ANNLinSig_MTBLS90.xlsx'

bootmodel.save_results(home + file)